# **(ADD THE NOTEBOOK NAME HERE)**

## Objectives

Create a single, analysis-ready dataset by:

* Loading and cleaning the two raw CSVs from the Global Food Price Inflation 2024 Kaggle archive

* Standardising column names, parsing dates, and coercing numeric fields

* Harmonising key categorical fields (e.g., country, item)

* Merging item-/series-level details with country-level context using reliable keys

* Saving cleaned per-file outputs and a merged dataset for downstream analysis

## Inputs

* data/raw/WLD_RTFP_country_2023-10-02.csv

* data/raw/WLD_RTP_details_2023-10-02.csv

* Python packages: pandas, numpy, os, re, pathlib

## Outputs

* data/processed/WLD_RTFP_country_2023-10-02_clean.csv

* data/processed/WLD_RTP_details_2023-10-02_clean.csv

* data/processed/food_price_merged_clean.csv (final, analysis-ready)

## Additional Comments

* Join strategy: left join from details → country, prioritising keys ['country','date','item'] then ['country','date'], falling back to ['country'] only if needed.

* If your columns differ slightly (e.g., country_name instead of country, month instead of date), the notebook will adapt and log exactly which keys were used.

* Currency/unit normalisation can be added as a follow-up step once business rules are agreed.



---

# Change working directory

* We assume this notebook sits in a subfolder (e.g., jupyter_notebooks/). We make the parent the working directory.

In [1]:
# Access the current directory
import os
current_dir = os.getcwd()
current_dir

'/Users/aminaibrahim/Documents/vscode-projects/food-price-inflation-analysis/jupyter_notebooks'

In [2]:
# Make the parent directory current
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [3]:
# Confirm new current directory
current_dir = os.getcwd()
current_dir

'/Users/aminaibrahim/Documents/vscode-projects/food-price-inflation-analysis'

# Section 1

Section 1 content

In [4]:
import pandas as pd

# Load dataset
country_path = "data/WLD_RTFP_country_2023-10-02.csv"
details_path = "data/WLD_RTP_details_2023-10-02.csv"

country_df = pd.read_csv(country_path)
details_df = pd.read_csv(details_path)

---

# Section 2

## Data Cleaning Utility Functions

The following code defines a set of reusable functions to clean and standardize tabular data using pandas. These functions help ensure consistent column naming, tidy string values, parse dates, convert numeric columns, drop nearly empty columns, and provide a quick summary report. You can use these utilities to prepare raw datasets for analysis.

In [5]:
import re
import numpy as np
import pandas as pd

# Standardize column names: strip spaces, remove special characters, replace spaces with underscores, and lowercase
def normalise_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = (
        df.columns
          .str.strip()
          .str.replace(r"[^\w\s]", "", regex=True)
          .str.replace(r"\s+", "_", regex=True)
          .str.lower()
    )
    return df

# Clean string columns: strip leading/trailing spaces and collapse multiple spaces into one
def tidy_strings(df: pd.DataFrame, cols) -> pd.DataFrame:
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[c] = (df[c].astype(str)
                           .str.strip()
                           .str.replace(r"\s+", " ", regex=True))
    return df

# Parse columns with names like 'date', 'month', 'year', or 'period' into datetime objects
def parse_dates(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if re.search(r"(date|month|year|period)", c):
            try:
                df[c] = pd.to_datetime(df[c], errors="coerce")
            except Exception:
                pass
    return df

# Convert columns that look numeric (even if stored as strings) to numeric dtype
def coerce_numeric(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if df[c].dtype == object:
            sample = df[c].dropna().astype(str).head(50)
            looks_numeric = (
                not sample.empty and
                sample.str.replace(",","", regex=False)
                      .str.replace("%","", regex=False)
                      .str.match(r"^-?\d+(\.\d+)?$")
                      .mean() > 0.6
            )
            if looks_numeric:
                df[c] = (df[c].astype(str)
                               .str.replace(",","", regex=False)
                               .str.replace("%","", regex=False)
                               .str.strip())
                df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

# Drop columns that are nearly empty (default: 98% or more missing values)
def drop_nearly_empty(df: pd.DataFrame, thresh=0.98) -> pd.DataFrame:
    na_ratio = df.isna().mean()
    to_drop = na_ratio[na_ratio >= thresh].index.tolist()
    return df.drop(columns=to_drop) if to_drop else df

# Run all cleaning steps in sequence for a generic DataFrame
def clean_generic(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out = normalise_columns(out)
    out = out.drop_duplicates()
    # Identify likely categorical columns and clean them
    cat_cols = [c for c in out.columns if out[c].dtype == object]
    priority = [c for c in ["country","country_name","item","commodity","product","unit","currency","series","market"] if c in out.columns]
    out = tidy_strings(out, list(dict.fromkeys(priority + cat_cols)))
    for c in ["country","country_name"]:
        if c in out.columns:
            out[c] = out[c].str.title()
    out = parse_dates(out)
    out = coerce_numeric(out)
    out = drop_nearly_empty(out, thresh=0.98)
    return out

# Print a brief summary report of the DataFrame's shape, duplicate count, and top missing columns
def brief_report(df: pd.DataFrame, name: str):
    print(f"{name}: shape={df.shape}, duplicates={df.duplicated().sum()}")
    miss = df.isna().sum()
    if miss.any():
        print("  top missing:", miss.sort_values(ascending=False).head(5).to_dict())

# Section 3

## Applying Data Cleaning Functions

Now we use the cleaning utilities to process both datasets (`country_df` and `details_df`).  
We then print a brief summary report for each cleaned DataFrame to check their shape, duplicate count, and missing values.

In [6]:
# Clean both datasets using the generic cleaning function
country_clean = clean_generic(country_df)
details_clean = clean_generic(details_df)

# Print a summary report for each cleaned DataFrame
brief_report(country_clean, "Country (clean)")
brief_report(details_clean, "Details (clean)")

Country (clean): shape=(4798, 8), duplicates=0
  top missing: {'inflation': 364, 'open': 64, 'high': 64, 'low': 64, 'close': 64}
Details (clean): shape=(25, 21), duplicates=0


/var/folders/09/yd5kcn1539g31p80cnw4dmr80000gn/T/ipykernel_39963/3821305012.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")
/var/folders/09/yd5kcn1539g31p80cnw4dmr80000gn/T/ipykernel_39963/3821305012.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")
/var/folders/09/yd5kcn1539g31p80cnw4dmr80000gn/T/ipykernel_39963/3821305012.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[c] = pd.to_datetime(df[c], errors="coerce")
/var/folders/09/yd5kcn1539g31p80cnw4dmr80000gn/T/ipy

# Section 4 

## Saving Cleaned Data

After cleaning, we save the processed DataFrames as new CSV files in the `data/` directory.  
This ensures that the cleaned datasets are available for downstream analysis or sharing, without overwriting the original raw files.  
The file names include `_clean` to indicate they have been processed.

In [8]:
# Save cleaned versions alongside the raw files in 'data/'
country_out = "data/WLD_RTFP_country_2023-10-02_clean.csv"
details_out = "data/WLD_RTP_details_2023-10-02_clean.csv"

country_clean.to_csv(country_out, index=False)
details_clean.to_csv(details_out, index=False)

print("Saved:", country_out)
print("Saved:", details_out)

Saved: data/WLD_RTFP_country_2023-10-02_clean.csv
Saved: data/WLD_RTP_details_2023-10-02_clean.csv


# Section 5 — Merge to a single merged cleaned dataset

## Merging Cleaned Datasets

To combine the cleaned detail and country datasets, we first ensure both have consistent key columns for merging:
- `add_country_key` copies the `country_name` column to `country` if `country` is missing.
- `ensure_date` creates a `date` column from alternatives like `month`, `period`, or `year` if needed.

We then select the best available join keys from a prioritized list (e.g., `["country","date","item"]`), and perform a left merge from the details to the country data.  
This approach ensures robust merging even if column names or structures differ slightly between files.  
Finally, we print the merge keys used, the shape of the merged DataFrame, and the number of rows with missing `country` after merging.

In [9]:
def add_country_key(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "country" not in df.columns and "country_name" in df.columns:
        df["country"] = df["country_name"]
    return df

def ensure_date(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if "date" not in df.columns:
        for alt in ["month","period","year"]:
            if alt in df.columns and pd.api.types.is_datetime64_any_dtype(df[alt]):
                df["date"] = df[alt]
                break
    return df

country_keyed = ensure_date(add_country_key(country_clean))
details_keyed = ensure_date(add_country_key(details_clean))

candidate_key_sets = [
    ["country","date","item"],
    ["country","date"],
    ["country"]
]

chosen = None
for keys in candidate_key_sets:
    if all(k in country_keyed.columns for k in keys) and all(k in details_keyed.columns for k in keys):
        chosen = keys
        break

if not chosen:
    raise ValueError("No suitable join keys found. Inspect columns and adjust `candidate_key_sets`.")

print("Merging on keys:", chosen)

merged = pd.merge(
    details_keyed,
    country_keyed,
    on=chosen,
    how="left",
    suffixes=("", "_country")
).drop_duplicates()

print("Merged shape:", merged.shape)
no_match = merged["country"].isna().sum() if "country" in merged.columns else 0
print("Rows with missing `country` after merge:", int(no_match))

Merging on keys: ['country']
Merged shape: (4798, 28)
Rows with missing `country` after merge: 0


In [ ]:
# Check for duplicate keys before merging
print("Country duplicate keys:", country_keyed.duplicated(subset=chosen).sum())
print("Details duplicate keys:", details_keyed.duplicated(subset=chosen).sum())

Country duplicate keys: 4773
Details duplicate keys: 0


### Why I Need to Fix Duplicate Keys Before Merging

When I see duplicate keys in my `country_keyed` DataFrame, it means that for some join keys, there are multiple matching rows. If I merge without fixing this, each row in `details_keyed` will join with all matching rows in `country_keyed`, which can blow up my merged dataset and introduce errors.

To keep my merged data clean and accurate, I need to investigate and resolve these duplicates in `country_keyed` before merging. This might mean dropping exact duplicates, aggregating values, or deciding which record to keep for each key.

In [11]:
# Check for duplicate keys before merging
print("Country duplicate keys:", country_keyed.duplicated(subset=chosen).sum())
print("Details duplicate keys:", details_keyed.duplicated(subset=chosen).sum())

# Check for missing join keys
for k in chosen:
    print(f"Missing '{k}' in details:", details_keyed[k].isna().sum())
    print(f"Missing '{k}' in country:", country_keyed[k].isna().sum())

Country duplicate keys: 4773
Details duplicate keys: 0
Missing 'country' in details: 0
Missing 'country' in country: 0


## Resolving Duplicate Keys Before Merging

When I check for duplicate keys before merging, I see that there are 4773 duplicate key rows in my `country_keyed` DataFrame and 0 in `details_keyed`. This means that, for my chosen join keys, the country-level data is not unique. If I merge as-is, each row in `details_keyed` will join with all matching rows in `country_keyed`, which could inflate my merged dataset and introduce errors.

Additionally, I see there are no missing values for the 'country' key in either DataFrame, so I don't need to worry about missing join keys.

Now that I've identified duplicate keys in my `country_keyed` DataFrame, I need to resolve them before merging. This helps prevent data inflation and ensures the merged dataset is accurate.

If the duplicate rows are exact matches, I can safely drop them. If they differ, I'll need to review and decide how to aggregate or select a representative row for each key.

In [12]:
# Drop exact duplicate rows based on the chosen join keys
country_keyed = country_keyed.drop_duplicates(subset=chosen)

# Re-check for duplicates to confirm resolution
print("Country duplicate keys after cleaning:", country_keyed.duplicated(subset=chosen).sum())

Country duplicate keys after cleaning: 0


# Section 6 — Save merged cleaned file & quick Quality Assurance

## Section 6 — Save Merged Cleaned File & Quick Quality Assurance

Now that I've resolved duplicate keys and completed the merge, I will save the final cleaned dataset to disk.  
I'll also run a quick quality assurance check by summarizing numeric columns and showing the most frequent values for key categorical columns.  
This helps ensure the merged data is ready for downstream analysis and gives me a first look at the data distribution.

In [13]:
# Save the merged cleaned dataset
merged_out = "data/food_price_merged_cleaned.csv"
merged.to_csv(merged_out, index=False)
print("Saved merged cleaned dataset:", merged_out)

# Quick numeric summary (first 10 rows of summary only to keep output tidy)
import numpy as np
num_summary = merged.select_dtypes(include=[np.number]).describe().T.head(10)
display(num_summary)

# Frequent categories to guide EDA
for key in ["country", "item", "unit", "currency", "series"]:
    if key in merged.columns:
        print(f"\nTop values for `{key}`:")
        display(merged[key].value_counts().head(10))

Saved merged cleaned dataset: data/food_price_merged_cleaned.csv


,count,mean,std,min,25%,50%,75%,max
number_of_markets_modeled,4798.0,57.067945,46.429370,9.00,24.000,42.00,79.00,228.00
number_of_markets_covered,4798.0,57.067945,46.429370,9.00,24.000,42.00,79.00,228.00
number_of_food_items,4798.0,8.946644,6.629954,3.00,4.000,7.00,11.00,26.00
data_coverage_food,4798.0,34.872926,17.070135,8.84,20.125,31.08,47.97,69.91
average_annualized_food_inflation,4798.0,11.857870,14.511799,1.24,3.580,6.68,10.79,55.30
maximum_food_drawdown,4798.0,-23.375060,10.914257,-40.67,-32.670,-23.76,-13.96,-2.79
average_annualized_food_volatility,4798.0,10.614506,5.290399,1.84,7.150,9.89,12.58,24.77
average_annual_food_price_correlation_between_markets,4798.0,0.781647,0.171540,0.27,0.670,0.82,0.91,0.99
index_confidence_score,4798.0,0.919329,0.039085,0.82,0.890,0.93,0.95,0.99
open,4734.0,1.491880,4.652457,0.01,0.740,0.96,1.10,102.46



Top values for `country`:


country
Afghanistan    202
Somalia        202
Nigeria        202
Niger          202
Mozambique     202
Mali           202
Liberia        202
Burundi        202
Lao Pdr        202
South Sudan    202
Name: count, dtype: int64


Top values for `currency`:


currency
XOF    808
XAF    760
AFN    202
SOS    202
CDF    202
GMD    202
HTG    202
SSP    202
LAK    202
BIF    202
Name: count, dtype: int64

# Section 7 — Feature Engineering


Now that I have a clean, merged dataset, I can begin feature engineering to create new variables that may improve analysis and modeling. Feature engineering helps extract more information from the data and can reveal important trends or relationships.

**Possible feature engineering steps:**
- Create time-based features (e.g., year, month, quarter) from date columns.
- Calculate percentage change or rolling averages for price or inflation columns.
- Encode categorical variables (e.g., one-hot encoding for `country`, `item`, or `unit`).
- Create flags for missing or extreme values.
- Aggregate data at different levels (e.g., country-level monthly averages).

Below, I demonstrate some initial feature engineering steps.

In [ ]:
# Example feature engineering on the merged dataset

# 1. Extract year and month from a 'date' column if present
if "date" in merged.columns:
    merged["year"] = merged["date"].dt.year
    merged["month"] = merged["date"].dt.month

# 2. Calculate percentage change for a price column if present
if "close" in merged.columns:
    merged = merged.sort_values(["country", "date"])
    merged["close_pct_change"] = merged.groupby("country")["close"].pct_change()

# 3. Create a rolling average for 'close' price (window=3 months)
if "close" in merged.columns:
    merged["close_rolling_avg_3"] = merged.groupby("country")["close"].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# 4. One-hot encode the 'item' column if present
if "item" in merged.columns:
    merged = pd.get_dummies(merged, columns=["item"], prefix="item")

# 5. Flag missing values in 'inflation' column
if "inflation" in merged.columns:
    merged["inflation_missing"] = merged["inflation"].isna().astype(int)

# Preview the engineered features
display(merged.head())

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [7]:
import os
try:
  # create your folder here
  # os.makedirs(name='')
except Exception as e:
  print(e)


IndentationError: expected an indented block after 'try' statement on line 2 (553063055.py, line 5)